In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
#from modules.models import MyRNNUAVClass
#from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

eval_env = MyEnv()
policies = {
    #"blue_1": PolicySpec(config={"gamma": 0.99}),
    #"blue_2": PolicySpec(config={"gamma": 0.95}),
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": True,"use_lstm": True,"max_seq_len": 200},}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": True,"use_lstm": True,"max_seq_len": 200},}),
    #"blue_0": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    #"blue_1": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
}
policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config = ppo.DEFAULT_CONFIG.copy()
config = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 0,
          "train_batch_size": 600*10*5,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          #"clip_actions":True,"normalize_actions":True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "explore":True,
          "sgd_minibatch_size": 300, "num_sgd_iter":20,
          "exploration_config": {"type": "StochasticSampling","random_timesteps":0}, #PPO デフォルト "random_timesteps":0
          #"model":{"fcnet_activation": "relu","fcnet_hiddens": [256, 256, 256],"post_fcnet_activation": "linear",
          #         "vf_share_layers": True,},#"linear","relu","tanh" "use_lstm":True,"lstm_cell_size":256,"max_seq_len":128
          
          #"model": {"custom_model": "my_model"},
          "multiagent": {"policies": policies,  "policy_mapping_fn": policy_mapping_fn}
         }
#res_name = "sgd"+str(config["sgd_minibatch_size"])+"sgd_num"+str(config["num_sgd_iter"])+"lr"+str(config["lr"])+"gamma"+str(config["gamma"])
res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    trainer.restore(checkpoint_path=model_path)

models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-03-13 14:39:54,700	INFO trainer.py:723 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2022-03-13 14:39:54,701	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by the mult

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar  7 21:50:01 2022

@author: Takumi
"""
eval_env.reset()
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')
        #print(pretty_print(results))
        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            trainer.restore(checkpoint_path=model_path)
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_1=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            action_dict1 = [0,0]
            rewards = {"blue_0":0,"blue_1":0}
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                #for j in range(eval_env.blue_num):
                    #if not eval_env.blue[j].hitpoint == 0:
                    #action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])
                    #action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j),
                    #                                                   clip_actions=True,explore=False)
                    #action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j))

                action_dict0 = trainer.compute_single_action(obs['blue_0'],
                                                             state=state_0,prev_action=None,prev_reward=None,
                                                             policy_id='blue_0',explore=False)
                action_dict1 = trainer.compute_single_action(obs['blue_1'],
                                                             state=state_1,prev_action=None,prev_reward=None,
                                                             policy_id='blue_1',explore=False)
                
                #action_dict0 = trainer.compute_single_action(obs['blue_0'],policy_id='blue_0')
                #action_dict1 = trainer.compute_single_action(obs['blue_1'],policy_id='blue_1')
                state_0 = action_dict0[1]
                state_1 = action_dict1[1]
                obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0[0], 'blue_1': action_dict1[0]})

                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])

                # plt.clf()

                # plt.subplots_adjust(left=-0.1,right=1.1,bottom=-0.1,top=1.1)
                # fig.canvas.draw()
                # plt.pause(.01)

                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()
                    
                #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)
                
                #env_blue_pos.append(env_blue_pos_temp)
                #env_red_pos.append(env_red_pos_temp)
                #env_mrm_pos.append(env_mrm_pos_temp)
                #step_num = step_num + 1
                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue",steps,env_blue_pos_mod)
                    save_hists("red",steps,env_red_pos_mod)
                    save_hists("mrm",steps,env_mrm_pos_mod)
                    # print(f'all done at {env.steps}')
                    break
                
            #del env_blue_pos[0]
            #del env_red_pos[0]
            #del env_mrm_pos[0]
            
            #hist_blue_pos = np.vstack(env_blue_pos)
            #hist_red_pos = np.vstack(env_red_pos)
            #hist_mrm_pos = np.vstack(env_mrm_pos)
            
            #f = open(results_file,'wb')
            #pickle.dump(emv_blue_pos,f)
            #pickle.dump(emv_red_pos,f)
            #pickle.dump(emv_mrm_pos,f)
            #f.close()
            
            #if record_mode == 0:
               # video.release()

ray.shutdown()

-------------------------- Scene: 0 --------------------------

----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 1 --------------------------
-------------------------- Scene: 0 --------------------------
50 blue_1 DOWN
349 blue_0 Shoot at red_1 launch distance : 58071.95961137256 True True
403 red_0 Shoot at blue_0
481 blue_0 Splash :red_1
483 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 483 -1.5244219836708663 10.373270716739853
blue_1 False False 483 -1.0100031170041996 -4.061084817472137
-------------------------- Scene: 1 --------------------------
44 blue_1 DOWN
349 blue_0 Shoot at red_0 launch distance : 59713.49471951403 True True
475 blue_0 Splash :red_0
TIME LIMIT LOSE
blue_0 False False 1200 3.0507285679226093e-06 2.6667971696524195
blue_1 False False 1200 -1.009996949271432 -4.051046726486945
-------------------------- Scene: 0 --------------------------
82 blue_0 DOWN
562 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False

485 blue_0 Splash :red_0
TIME LIMIT LOSE
blue_0 False False 1200 8.32783715283343e-07 -11.513141733505552
blue_1 False False 1200 -1.0099991672162847 -32.42354613386518
-------------------------- Scene: 1 --------------------------
65 blue_1 DOWN
229 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 229 -2.010001751207764 -4.050573302179908
blue_1 False False 229 -1.0100081512077639 -6.071410724398463
-------------------------- Scene: 0 --------------------------
364 blue_0 Shoot at red_1 launch distance : 59274.7229535375 True True
394 blue_1 DOWN
419 red_0 Shoot at blue_0
490 blue_0 Splash :red_1
518 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 518 -1.5243944288262907 -10.710540858196692
blue_1 False False 518 -1.010005262159624 -21.304713902760568
-------------------------- Scene: 0 --------------------------
105 blue_1 DOWN
349 blue_0 Shoot at red_1 launch distance : 59868.950659986534 True True
408 red_0 Shoot at blue_0
472 blue_0 Splash :red_1
507 blue_0: Destroyed
TIME LIMI

465 blue_0 Splash :red_0
1043 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 1200 1.8801943439471263e-06 -42.516483904020255
blue_1 False False 1200 -1.009998119805656 -815.8570913867361
-------------------------- Scene: 1 --------------------------


2022-03-13 15:24:44,652	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


agent_timesteps_total: 61736
custom_metrics: {}
date: 2022-03-13_15-36-45
done: false
episode_len_mean: 834.2702702702703
episode_media: {}
episode_reward_max: 7.482516911243372
episode_reward_mean: -143.62320306523424
episode_reward_min: -858.3735752907558
episodes_this_iter: 37
episodes_total: 37
experiment_id: b8187180aeca4c8e9b35c5422f551d66
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 6.605915069580078
        entropy_coeff: 0.0
        kl: 0.055880870670080185
        model: {}
        policy_loss: 0.014421229250729084
        total_loss: 28.450176239013672
        vf_explained_var: -0.13921049237251282
        vf_loss: 28.424579620361328
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 6.838758945465088
        entropy_coeff: 0.0
  

...

207 blue_1 DOWN
507 blue_0 Shoot at red_1 launch distance : 57812.30030691187 True True
563 red_0 Shoot at blue_0
631 blue_0 Splash :red_1
730 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 730 -1.5228282556987658 -19.337869159626667
blue_1 False False 730 -1.0099989223654324 -18.102124047192625
-------------------------- Scene: 0 --------------------------
85 blue_1 DOWN
90 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 90 -2.010004907889403 -5.0603727507407505
blue_1 False False 90 -1.0100084412227364 -5.071315115778206
-------------------------- Scene: 1 --------------------------
272 blue_1 DOWN
431 blue_0 Shoot at red_0 launch distance : 59003.26719076487 True True
560 blue_0 Splash :red_0
TIME LIMIT LOSE
blue_0 False False 1200 2.5903803399875563e-06 -14.757310103886883
blue_1 False False 1200 -1.00999740961966 -9.673059942327045
-------------------------- Scene: 0 --------------------------
122 blue_0 DOWN
365 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 365 -1.010007784

173 blue_1 DOWN
305 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 305 -2.010003974718722 -5.39219058793712
blue_1 False False 305 -1.0100080747187217 -4.102246646641499
-------------------------- Scene: 0 --------------------------
83 blue_1 DOWN
362 blue_0 Shoot at red_1 launch distance : 59977.186545629724 True True
404 red_0 Shoot at blue_0
482 blue_0 Splash :red_1
506 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 506 -1.5244581238653547 -19.469216376134188
blue_1 False False 506 -1.0100038238653548 -6.071701866513224
-------------------------- Scene: 0 --------------------------
172 blue_1 DOWN
312 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 312 -2.0100080227117765 -9.561472176675746
blue_1 False False 312 -1.0100081227117765 -4.762953039232132
-------------------------- Scene: 1 --------------------------
57 blue_1 DOWN
97 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 97 -2.0100027304029005 -4.050426853087144
blue_1 False False 97 -1.0100088637362339 -6.071228525597398

72 blue_1 DOWN
400 blue_0 Shoot at red_1 launch distance : 56795.57693003207 True True
451 red_0 Shoot at blue_0
517 blue_0 Splash :red_1
577 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 577 -1.5241907231290748 -6.694453233354261
blue_1 False False 577 -1.0100013897957414 -4.051605544756288
-------------------------- Scene: 0 --------------------------
55 blue_1 DOWN
500 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 500 -2.010012434592494 -12.085463798259887
blue_1 False False 500 -1.0100084012591608 -5.061080573213779
-------------------------- Scene: 1 --------------------------
131 blue_1 DOWN
297 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 297 -2.010010164068677 -6.881860428202044
blue_1 False False 297 -1.010008230735344 -5.061784458260296
-------------------------- Scene: 0 --------------------------
126 blue_1 DOWN
208 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 208 -2.010009718128848 -10.331233436609882
blue_1 False False 208 -1.0100082514621815 -6.12226941587275

2022-03-13 23:21:26,756	INFO trainable.py:468 -- Restored on 127.0.0.1 from checkpoint: ./UCAV/checkpoints/test_2\checkpoint_000009\checkpoint-9
2022-03-13 23:21:26,757	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 9, '_timesteps_total': 0, '_time_total': 30497.711319446564, '_episodes_total': 478}


agent_timesteps_total: 547378
custom_metrics: {}
date: 2022-03-13_23-21-26
done: false
episode_len_mean: 441.52
episode_media: {}
episode_reward_max: 10.955317030232743
episode_reward_mean: -29.899360261226562
episode_reward_min: -329.5857679490732
episodes_this_iter: 66
episodes_total: 478
experiment_id: b8187180aeca4c8e9b35c5422f551d66
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 5.125781059265137
        cur_lr: 0.0002500000118743628
        entropy: 7.826497554779053
        entropy_coeff: 0.0
        kl: 0.03583569824695587
        model: {}
        policy_loss: 0.008569608442485332
        total_loss: 18.16096305847168
        vf_explained_var: -0.050001829862594604
        vf_loss: 17.968708038330078
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 5.125781059265137
        cur_lr: 0.0002500000118743628
        entropy: 7.471248626708984
        entropy_coeff: 0.0
        kl: 0.1

#### 